<a href="https://colab.research.google.com/github/bastrob/Deep_learning_projects/blob/master/Sentiment_analysis_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf